Классификатор для распознавания рукописных букв русского алфавита
на основе метода опорных векторов.

In [ ]:
!pip install sklearn

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sn
from matplotlib import pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
from google.colab import drive
drive.mount('\content\gdrive')

In [ ]:
data = pd.read_csv('Letters_crosses.csv').drop('Unnamed: 0', axis=1)

In [ ]:
data['letter'].value_counts()

In [ ]:
data

In [ ]:
# Удалим из таблицы все векторы признаков, у которых все координаты нулевые:

limit = data.shape[0]

rows_to_delete = []
for row in range(limit):
    observation = list(data.iloc[row, : -1])
    all_zeros = 1
    for element in observation:
        if element != 0:
            all_zeros = 0
    if all_zeros == 1:
        rows_to_delete.append(row)

for row in rows_to_delete:
    data = data.drop(row)
    # print(f'Строчка из таблицы с данными: {row} удаляется.', end=' ')

data.reset_index(drop=True)

In [ ]:
data['letter'].value_counts()

In [ ]:
letter_label = {'a': 0, 'б': 1, 'в': 2, 'г': 3, 'д': 4, 'е': 5, 'ё': 6, 'ж': 7,
                 'з': 8, 'и': 9, 'й': 10, 'к': 11, 'л': 12, 'м': 13, 'н': 14,
                 'о': 15, 'п': 16, 'р': 17, 'с': 18, 'т': 19, 'у': 20, 'ф': 21,
                 'х': 22, 'ц': 23, 'ч': 24, 'ш': 25, 'щ': 26, 'ъ': 27, 'ы': 28,
                 'ь': 29, 'э': 30, 'ю': 31, 'я': 32}
label_letter = {0: 'a', 1: 'б', 2: 'в', 3: 'г', 4: 'д', 5: 'е', 6: 'ё', 7: 'ж',
                 8: 'з', 9: 'и', 10: 'й', 11: 'к', 12: 'л', 13: 'м', 14: 'н',
                 15: 'о', 16: 'п', 17: 'р', 18: 'с', 19: 'т', 20: 'у', 21: 'ф',
                 22: 'х', 23: 'ц', 24: 'ч', 25: 'ш', 26: 'щ', 27: 'ъ', 28: 'ы',
                 29: 'ь', 30: 'э', 31: 'ю', 32: 'я'}

In [ ]:
data['labels'] = data['letter'].apply(lambda x: letter_label[x])

In [ ]:
print(data.iloc[[5222, 3607]])

In [ ]:
letters_labels = data['labels']
letters_features = data.drop(['letter', 'labels'], axis=1)

In [ ]:
# Вначале с помощью train_test_split разобьем наши данные на обучающую
# и тестовую выборки:

x_train, x_test, y_train, y_test = train_test_split(letters_features, 
                                                    letters_labels,
                                                    train_size=0.7,
                                                    random_state=42,
                                                    stratify=letters_labels)

In [ ]:
# SVM классификатор.
svc = SVC()

In [ ]:
svc = svc.fit(x_train, y_train)

In [ ]:
y_pred = svc.predict(x_test)

In [ ]:
# Confusion matrix.
confusion_matrix_arr = confusion_matrix(y_test, y_pred)

In [ ]:
# Seaborn Heatmap
confusion_matrix_df = pd.DataFrame(confusion_matrix_arr)

fig, ax = plt.subplots(figsize=(18,10))
sn.heatmap(confusion_matrix_df, annot=True, fmt='d', ax=ax, annot_kws={"size": 14},
           xticklabels = ['a', 'б', 'в', 'г', 'д', 'е', 'ё', 'ж', 'з', 'и', 'й',
                          'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф',
                          'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я'],
           yticklabels = ['a', 'б', 'в', 'г', 'д', 'е', 'ё', 'ж', 'з', 'и', 'й',
                          'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф',
                          'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я'])

plt.savefig('Letters_crosses_SVM.png')
plt.show()

In [ ]:
# Метрики качества классификации: precision, recall, f1-score.
clf_rep = classification_report(y_test, y_pred)
print(clf_rep)